In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from math import radians, sin, cos, asin, sqrt
from nfl_data_py import import_schedules, import_weekly_data

# Optional: try FiveThirtyEight Elo
def _try_load_elo():
    try:
        url = "https://projects.fivethirtyeight.com/nfl-api/nfl_elo_latest.csv"
        elo = pd.read_csv(url)
        elo["season"] = elo["season"].astype(int)
        elo.rename(columns={"team1": "home_team", "team2": "away_team",
                            "elo1_pre": "elo_home", "elo2_pre": "elo_away"}, inplace=True)
        return elo[["season", "home_team", "away_team", "elo_home", "elo_away"]]
    except Exception:
        print("⚠️ Elo dataset unavailable — skipping Elo merge.")
        return pd.DataFrame(columns=["season", "home_team", "away_team", "elo_home", "elo_away"])

# Team home-market coordinates (for travel miles)
TEAM_COORDS = {
    "BUF": (42.7738, -78.7868), "MIA": (25.9580, -80.2389), "NE": (42.0909, -71.2643), "NYJ": (40.8136, -74.0744),
    "BAL": (39.2780, -76.6227), "CIN": (39.0955, -84.5161), "CLE": (41.5061, -81.6995), "PIT": (40.4468, -80.0158),
    "HOU": (29.6847, -95.4107), "IND": (39.7601, -86.1639), "JAX": (30.3239, -81.6373), "TEN": (36.1664, -86.7713),
    "DEN": (39.7439, -105.0201), "KC": (39.0489, -94.4839), "LV": (36.0908, -115.1830), "LAC": (33.8644, -118.2619),
    "DAL": (32.7473, -97.0945), "NYG": (40.8136, -74.0744), "PHI": (39.9008, -75.1675), "WAS": (38.9077, -76.8645),
    "CHI": (41.8623, -87.6167), "DET": (42.3390, -83.0456), "GB": (44.5013, -88.0622), "MIN": (44.9740, -93.2581),
    "ATL": (33.7554, -84.4008), "CAR": (35.2251, -80.8526), "NO": (29.9511, -90.0812), "TB": (27.9759, -82.5033),
    "ARI": (33.5276, -112.2626), "LAR": (34.0141, -118.2879), "SEA": (47.5952, -122.3316), "SF": (37.4030, -121.9700),
}

# ---------- UTILITIES ----------
def haversine_miles(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1,lon1,lat2,lon2])
    a = sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2)*sin((lon2-lon1)/2)**2
    return 3958.8 * 2 * asin(sqrt(a))

def _clean_weather(df):
    for col in ["temp", "wind"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    df["temp"] = df["temp"].fillna(df["temp"].mean())
    df["wind"] = df["wind"].fillna(0)
    df["cold_game"] = (df["temp"] < 40).astype(int)
    df["windy"] = (df["wind"] > 15).astype(int)
    return df

def _normalize_weekly(df):
    rename = {"season_year":"season","team_abbr":"team","recent_team":"team","season_type":"game_type"}
    df = df.rename(columns={k:v for k,v in rename.items() if k in df.columns})
    for c in ["season","week","team","player_id"]:
        if c not in df.columns: df[c]=np.nan
    return df

def _team_week_agg(weekly):
    weekly=_normalize_weekly(weekly)
    rename={"rush_yds":"rushing_yards","pass_yds":"passing_yards","fantasy_points":"points"}
    weekly=weekly.rename(columns=rename)
    cols=[c for c in ["rushing_yards","passing_yards","points","field_goals_made"] if c in weekly.columns]
    agg=weekly.groupby(["team","season","week"],as_index=False)[cols].sum(min_count=1)
    active=weekly.groupby(["team","season","week"])["player_id"].nunique().reset_index(name="active_players")
    starters=pd.DataFrame(columns=["team","season","week","starter_ids"])
    if "starter" in weekly.columns:
        starters=(weekly[weekly["starter"]==True]
                  .groupby(["team","season","week"])["player_id"]
                  .apply(lambda x: ",".join(sorted(set(map(str,x))))).reset_index(name="starter_ids"))
    out=agg.merge(active,on=["team","season","week"],how="left").merge(starters,on=["team","season","week"],how="left")
    return out

def _rolling_avgs(team_df):
    team_df=team_df.sort_values(["team","season","week"])
    for c in ["rushing_yards","passing_yards","points","field_goals_made"]:
        if c in team_df.columns:
            team_df[f"avg_{c}"]=(team_df.groupby(["team","season"])[c]
                                 .expanding().mean().reset_index(level=[0,1],drop=True))
    return team_df

def _rest_days(g):
    g=g.copy()
    g["gameday"]=pd.to_datetime(g["gameday"],errors="coerce")
    long=pd.concat([
        g[["game_id","season","week","gameday","home_team"]].rename(columns={"home_team":"team"}),
        g[["game_id","season","week","gameday","away_team"]].rename(columns={"away_team":"team"})
    ])
    long=long.sort_values(["team","season","week"])
    long["prev_gameday"]=long.groupby("team")["gameday"].shift(1)
    long["rest_days"]=(long["gameday"]-long["prev_gameday"]).dt.days.fillna(9)
    long["short_week"]=(long["rest_days"]<7).astype(int)
    home=long.merge(g[["game_id","home_team"]],left_on=["game_id","team"],right_on=["game_id","home_team"],how="inner")
    away=long.merge(g[["game_id","away_team"]],left_on=["game_id","team"],right_on=["game_id","away_team"],how="inner")
    home=home.rename(columns={"rest_days":"home_rest_days","short_week":"home_short_week"})[["game_id","home_rest_days","home_short_week"]]
    away=away.rename(columns={"rest_days":"away_rest_days","short_week":"away_short_week"})[["game_id","away_rest_days","away_short_week"]]
    return g[["game_id"]].merge(home,on="game_id",how="left").merge(away,on="game_id",how="left")

def _travel(g):
    g=g.copy()
    def coord(t): return TEAM_COORDS.get(t,(np.nan,np.nan))
    g["home_lat"],g["home_lon"]=zip(*g["home_team"].map(coord))
    g["away_lat"],g["away_lon"]=zip(*g["away_team"].map(coord))
    g["home_travel_miles"]=0
    g["away_travel_miles"]=g.apply(
        lambda r: haversine_miles(r["away_lat"],r["away_lon"],r["home_lat"],r["home_lon"])
        if pd.notnull(r["home_lat"]) and pd.notnull(r["away_lat"]) else np.nan,axis=1)
    return g[["game_id","home_travel_miles","away_travel_miles"]]

# ---------- MAIN BUILDER ----------
def build_nfl_dataset(start_year=2015,end_year=2025,outfile="nfl_game_results_full_model.xlsx"):
    elo_df=_try_load_elo()
    all=[]
    for season in range(start_year,end_year+1):
        print(f"\n🏈 Fetching season {season}...")
        try:
            s=import_schedules([season])
            g=s[(s["game_type"]=="REG")&(s["home_score"].notna())].copy()
            if g.empty: continue
            g["game_datetime"]=pd.to_datetime(g["gameday"],errors="coerce",utc=True)
            g=_clean_weather(g)
            g["score_diff"]=g["home_score"]-g["away_score"]
            g["home_win"]=(g["score_diff"]>0).astype(int)
            g["away_win"]=(g["score_diff"]<0).astype(int)
            g["home_favored"]=(g.get("spread_line",pd.Series(0))<0).astype(int)
            g["expected_margin"]=-g.get("spread_line",pd.Series(np.nan))
            g["venue_code"]=pd.factorize(g.get("stadium",pd.Series()))[0]

            # Add Elo if available
            if not elo_df.empty:
                elo_season=elo_df[elo_df["season"]==season]
                g=g.merge(elo_season,on=["season","home_team","away_team"],how="left")
                if "elo_home" in g.columns and "elo_away" in g.columns:
                    g["elo_diff"]=g["elo_home"]-g["elo_away"]
                    g["elo_avg"]=g[["elo_home","elo_away"]].mean(axis=1)

            w=import_weekly_data([season])
            w=_team_week_agg(w)
            w=_rolling_avgs(w)
            for side in ["home","away"]:
                g=g.merge(w.add_prefix(f"{side}_"),
                          left_on=[f"{side}_team","season","week"],
                          right_on=[f"{side}_team",f"{side}_season",f"{side}_week"],how="left")

            g=g.merge(_rest_days(g),on="game_id",how="left")
            g=g.merge(_travel(g),on="game_id",how="left")
            g=g.loc[:,~g.columns.duplicated()]
            all.append(g)
            print(f"✅ Season {season} OK ({len(g)} games)")
        except Exception as e:
            print(f"⚠️ Skipped {season}: {e}")

    if not all: raise RuntimeError("No valid data collected.")

    df=pd.concat(all,ignore_index=True)
    df["game_datetime"]=pd.to_datetime(df["game_datetime"],errors="coerce").dt.tz_localize(None)
    df=df.dropna(axis=1,how="all")
    df.to_excel(outfile,index=False)
    print(f"\n✅ Saved: {outfile}")
    print(f"Rows: {len(df)}, Columns: {len(df.columns)}")
    print("Columns:", df.columns[:25].tolist())
    return df

if __name__=="__main__":
    build_nfl_dataset(start_year=2015,end_year=2025)


⚠️ Elo dataset unavailable — skipping Elo merge.

🏈 Fetching season 2015...


C:\Users\moham\AppData\Local\Temp\ipykernel_1872\1700319773.py:123: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  g["venue_code"]=pd.factorize(g.get("stadium",pd.Series()))[0]


Downcasting floats.
✅ Season 2015 OK (256 games)

🏈 Fetching season 2016...


C:\Users\moham\AppData\Local\Temp\ipykernel_1872\1700319773.py:123: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  g["venue_code"]=pd.factorize(g.get("stadium",pd.Series()))[0]


Downcasting floats.
✅ Season 2016 OK (256 games)

🏈 Fetching season 2017...


...

Downcasting floats.
✅ Season 2023 OK (272 games)

🏈 Fetching season 2024...


C:\Users\moham\AppData\Local\Temp\ipykernel_1872\1700319773.py:123: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  g["venue_code"]=pd.factorize(g.get("stadium",pd.Series()))[0]


Downcasting floats.
✅ Season 2024 OK (272 games)

🏈 Fetching season 2025...


C:\Users\moham\AppData\Local\Temp\ipykernel_1872\1700319773.py:123: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  g["venue_code"]=pd.factorize(g.get("stadium",pd.Series()))[0]


⚠️ Skipped 2025: HTTP Error 404: Not Found

✅ Saved: nfl_game_results_full_model.xlsx
Rows: 2623, Columns: 79
Columns: ['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday', 'gametime', 'away_team', 'away_score', 'home_team', 'home_score', 'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis', 'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest', 'away_moneyline']
